# 第四章 使用树搜索下其
- 用极小极大算法找到最佳动作
- 对极小化极大树搜索剪枝，加快速度
- 将蒙特卡洛树搜索应用于游戏

在计算机科学中，`树搜索算法`是一类搜索策略，这些算法循环遍历许多可能的决策序列，并找到那些能产生最佳结果的决策。

## 4.1 游戏分类
- 确定与不确定
- 完全透明和隐藏信息

## 4.2 利用极小极大法搜索树预测对手
我们期望自己的得分最大化，二对手则希望我们的得分最小化。

代码清单4-1 找到可以立刻赢得比赛的动作

In [1]:
def find_winning_move(game_state, next_player):
    for candidate_move in game_state.legal_moves(next_player):
        next_state = game_state.apply_move(candidate_move)
        if next_state.is_over() and next_state.winner == next_player():
            return candidate_move # This move will win, no need to seach again
        return None # Can't win in this round.

代码清单 4-2: 避免让对手直接获胜的函数

In [2]:
def eliminate_losing_moves(game_state, next_player):
    opponent = next_player.other()
    possible_moves = []
    for candidate_move in game_state.legal_moves(next_playe):
        next_state = game_state.apply_move(candidate_move)
        opponent_winning_move = find_winning_move(next_state, opponent)
        if opponent_winning_move is None:
            possible_moves.append(candidate_move)
    return possible_moves

尝试寻找一个动作，让对手无法阻止我们后续的获胜动作。

代码清单 4-3 找到两步棋之后可以保证获胜的函数

In [4]:
def find_two_step_win(game_state, next_player):
    oponent = next_player.other()
    for candidate_move in game_state.legal_moves(next_player):
        next_state = game_state.apply_move(candidate_move)
        good_responses = eliminating_losing_move(next_state, opponent)
    if not good_responses:
        return candidate_move
    return None

通用策略的雏形
1. 先检查是否可以在下一步直接获胜，如果可以，就这样行动。
2. 如果不可以，在看看对手能否在下一步获胜，如果能，尝试阻止它
3. 如果对手不能获胜，再看看能否通过第二步棋获胜，如果能，就按这两步棋落子
4. 如果不能，再看对手的第二步棋是否能获胜

## 井字棋推演：极大极小算法的例子

首先定义表示棋局结果的三种可能

In [6]:
import enum
class GameResult(enum.Enum):
    loss = 1
    draw = 2
    win = 3

假设有个函数`best_result`， 可以给出某个游戏转台之后棋手能够获得的最佳结果，那么可以轻松地写一个函数在回合中选择动作：遍历所有可能的动作，逐一调用`best_result`， 然后选择其中最佳的动作即可。如果有多个动作能得到相同结果，则随机选择一个

In [8]:
class MinMaxAgent(agent):
    def select_move(self, game_state):
        wining_moves = []
        draw_moves = []
        losing_moves = []
        for possible_move in game_state.legal_moves():
            next_state = game_state.apply_move(possible_move)
            opponent_best_outcome = best_result(next_state)
            our_best_outcome = reverse_game_result(opponent_best_outcome)
            if our_best_outcome == GameResult.win:
                wining_moves.append(possible_move)
            elif our_best_outcome == GameResult.draw:
                draw_moves.append(possible_move)
            else:
                losing_moves.append(possible_move)
        
        if winning_moves:
            return random.choice(winning_moves)
        if draw_moves:
            return random.choice(draw)
        return random.choice(losing_moves)


NameError: name 'agent' is not defined

极小极大搜索的第一步：

In [9]:
def best_result(game_state):
    if game_state.is_over():
        if game_state.winner() == game_state.next_player:
            return gameResult.win
        elif game_state.winner() is None:
            return GameResult.draw
        else: 
            return GameResult.loss
        
    best_result_so_far = GameResult.loss
    for candidate_move in game_state.legal_moves():
        next_state = game_state.apply_move(candidate_move)
        opponent_best_result = best_result(next_state)
        our_result = reverse_game_result(opponent_best_result)
        if our_result.value > best_result_so_far.value:
            best_result_so_far = our_result
        return best_result_so_far

SyntaxError: invalid syntax (3293049555.py, line 3)

如果将这个算法用于类似井字棋这样的简单游戏，就会得到一个不会失败的棋手。

## 通过剪枝算法缩减搜索空间

两种剪枝技术：用于减少搜索深度的*棋局评估函数*，另一种是用与减少搜索跨度的$\alpha \beta$*剪枝*。
这两项技术构成了经典棋盘游戏AI的支柱。